In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
'''
This cell will print data for each file in the specified folder in its own tables.
Alternately, just specify the single file directly.
'''

import re
import matplotlib.pyplot as plt

import glob
files = glob.glob("/content/gdrive/My Drive/Colab Notebooks/Summer 2019 ARNI Research/*.txt")

summarydata = re.compile(r'Summary info:')
epochdata = re.compile(r'Epoch ')
acctraindata = re.compile(r'training')
acctestunrounddata = re.compile(r'without rounding')
acctestrounddata = re.compile(r'with rounding')
acctestdata = re.compile(r'test images')
nums = re.compile(r'[\d.]+')

for afile in files:
    print(afile)

    epochlist = []
    losslist = []
    cellosslist = []
    mselosslist = []

    acctrainlist = []
    acctestlist = []
    accunroundlist = []
    accroundlist = []

    with open (afile, 'rt') as file:
        output = []
        for line in file:
            if (summarydata.search(line)):
                print(line)
            elif (epochdata.search(line)):
                vals = nums.findall(line)
                if (len(vals) == 10):
                    _, _, _, epoch, _, stepval, stepnum, loss, cel, mse = nums.findall(line)                
                    cellosslist.append(float(cel))
                    mselosslist.append(float(mse))
                else:
                    _, _, _, epoch, _, stepval, stepnum, loss = nums.findall(line) 
                epochlist.append(float(epoch) - 1 + float(stepval)/float(stepnum))
                losslist.append(float(loss))
            elif (acctraindata.search(line)):
                acctrainlist.append(float(nums.findall(line)[0]))
            elif (acctestunrounddata.search(line)):
                accunroundlist.append(float(nums.findall(line)[1]))
            elif (acctestrounddata.search(line)):
                accroundlist.append(float(nums.findall(line)[1]))
            elif (acctestdata.search(line)):
                acctestlist.append(float(nums.findall(line)[1]))
            elif (len(line) > 17):
                print(line[16:], end = '')
                
    if (len(acctestlist) > 0):               
        accskip = len(epochlist) // len(acctestlist)
        fig= plt.figure(figsize=(4,3))
        plt.plot(epochlist, losslist, label = 'Loss')
    #    if(len(cellosslist) > 0):
   #         plt.plot(epochlist, cellosslist, label = 'Cross Entropy Loss only')
   #         plt.plot(epochlist, mselosslist, label = 'MSE Loss from rounding only')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.title('1 Channel Conv, no batchnorm, CEL only: Sample Loss')
        plt.legend(loc = 1)
        plt.show()
        fig= plt.figure(figsize=(4,3))
        plt.plot(epochlist[:len(acctestlist)*accskip:accskip], acctrainlist[:len(acctestlist)], label = 'train')
        plt.plot(epochlist[:len(acctestlist)*accskip:accskip], acctestlist[:len(acctestlist)], label = 'test rounded')
        plt.ylabel('Percent Accuracy')
        plt.xlabel('Epoch') 
        plt.title('1 Channel Conv, no batchnorm, CEL only: Accuracy Rates')
        plt.legend(loc = 'best')
        plt.show()
    elif(len(accunroundlist) > 0):
        accskip = len(epochlist) // len(accunroundlist)
        fig= plt.figure(figsize=(4,3))
        plt.plot(epochlist, losslist, label = 'Loss')
        if(len(cellosslist) > 0):
            plt.plot(epochlist, cellosslist, label = 'Cross Entropy Loss only')
            plt.plot(epochlist, mselosslist, label = 'MSE Loss from rounding only')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.title('3 Channel, CEL only, 4 Bit: Sample Loss')
        plt.legend(loc = 1)
        plt.show()
        fig= plt.figure(figsize=(4,3))
        plt.plot(epochlist[:len(accroundlist)*accskip:accskip], acctrainlist[:len(accroundlist)], label = 'train')
        plt.plot(epochlist[:len(accroundlist)*accskip:accskip], accunroundlist[:len(accroundlist)], label = 'test unrounded')
        plt.plot(epochlist[:len(accroundlist)*accskip:accskip], accroundlist[:len(accroundlist)], label = 'test rounded')
#        plt.ylim(0,100)
        plt.ylabel('Percent Accuracy')
        plt.xlabel('Epoch')        
        plt.title('3 Channel, CEL only, 4 Bit: Accuracy Rates')
        plt.legend(loc = 'best')
        plt.show()

In [7]:
'''
This cell prints all accuracy rates for all trials in the specified folder to one table.
'''

import re
import matplotlib.pyplot as plt

import glob
files = glob.glob("/content/gdrive/My Drive/Colab Notebooks/Summer 2019 ARNI Research/*.txt")

epochdata = re.compile(r'Epoch ')
acctraindata = re.compile(r'training')
acctestunrounddata = re.compile(r'without rounding')
acctestrounddata = re.compile(r'with rounding')
acctestdata = re.compile(r'test images')
nums = re.compile(r'[\d.]+')
filename = re.compile(r'[\d\-a-zA-Z]*.txt')

# These may be used for legend labels, see comment section below
#losstype = re.compile(r'Loss function:  ')
#multiplier = re.compile(r'MSE multiplier:  ')
#encoder = re.compile(r'Encoder type:  ')
#encoders = re.compile(r'Encoders:  ')
#bitsize = re.compile(r'Bits per encoder:  ')

fig= plt.figure(figsize=(15,6))
for afile in files:
    f = filename.search(afile)
    filelabel = f.group(0)

    epochlist = []
    losslist = []
    cellosslist = []
    mselosslist = []

    acctrainlist = []
    acctestlist = []
    accunroundlist = []
    accroundlist = []

    with open (afile, 'rt') as file:
        output = []
        for line in file:
            if (epochdata.search(line)):
                vals = nums.findall(line)
                if (len(vals) == 10):
                    _, _, _, epoch, _, stepval, stepnum, loss, cel, mse = nums.findall(line)                
                    cellosslist.append(float(cel))
                    mselosslist.append(float(mse))
                else:
                    _, _, _, epoch, _, stepval, stepnum, loss = nums.findall(line) 
                epochlist.append(float(epoch) - 1 + float(stepval)/float(stepnum))
                losslist.append(float(loss))
            elif (acctraindata.search(line)):
                acctrainlist.append(float(nums.findall(line)[0]))
            elif (acctestunrounddata.search(line)):
                accunroundlist.append(float(nums.findall(line)[1]))
            elif (acctestrounddata.search(line)):
                accroundlist.append(float(nums.findall(line)[1]))
            elif (acctestdata.search(line)):
                acctestlist.append(float(nums.findall(line)[1]))
# Below is a variety of options which can be used in place of the filelabel variable
# but it gets messy pretty quickly. Modify as needed, if needed
#            elif (len(line) > 17):
#                lossl = losstype.search(line)
#                mult = multiplier.search(line)
#                bits = bitsize.search(line)
#                enc = encoder.search(line)
#                encs = encoders.search(line)
#                if(lossl):
#                    thislabel = line[lossl.end():]
#                elif(mult):
#                    thislabel += ' Multiplier: ' + str(line[mult.end():])
#                elif(enc):
#                    thislabel += ' Encoder: ' + str(line[enc.end():])
#                elif(encs):
#                    thislabel += ' Num Encoders: ' + str(line[encs.end():])
#                elif(bits):
#                    thislabel += ' Bits: ' + str(line[bits.end():])
                    

    if (len(acctestlist) > 0):               
        accskip = len(epochlist) // len(acctestlist)
        plt.plot(epochlist[:len(acctestlist)*accskip:accskip], acctrainlist[:len(acctestlist)], label = 'train' + filelabel)
        plt.plot(epochlist[:len(acctestlist)*accskip:accskip], acctestlist[:len(acctestlist)], label = 'test rounded' + filelabel)
        
    elif(len(accunroundlist) > 0):
        accskip = len(epochlist) // len(accunroundlist)
        plt.plot(epochlist[:len(accroundlist)*accskip:accskip], acctrainlist[:len(accroundlist)], label = 'train' + filelabel)
        plt.plot(epochlist[:len(accroundlist)*accskip:accskip], accunroundlist[:len(accroundlist)], label = 'test unrounded' + filelabel)
        plt.plot(epochlist[:len(accroundlist)*accskip:accskip], accroundlist[:len(accroundlist)], label = 'test rounded' + filelabel)                  
                    
    plt.title('Train and Test Accuracy Rates')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

<Figure size 1080x432 with 0 Axes>